<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab10/GPT-pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab10

Mounted at /content/drive
/content/drive/MyDrive/ai-intensive2
Already up to date.
/content/drive/MyDrive/ai-intensive2/lab10


In [2]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# 11/17 Lab #9

1. GPT 사전학습 해보기
2. Instruction Tuning


## 인과적 언어 모델(Causal Language Model)

![](https://velog.velcdn.com/images%2Fnawnoes%2Fpost%2F17a78de6-6d65-4a76-9563-afaf05995afc%2Fimage.png)

사진처럼, 이전 토큰들을 받았을 때 때 다음 토큰을 예측하는 모델을 인과적 언어모델이라고 부릅니다.


![](https://static.packt-cdn.com/products/9781788834131/graphics/Images/B09475_06_04.jpg)
위 사진은 Character-level RNN을 이용한 예시입니다.



![](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc820b36a-8e07-4eea-8975-d7e391006d52_824x690.png)

트랜스포머 아키텍처에서는 우측 Decoder가 생성을 담당합니다. GPT는 Transformer에서 decode만을 사용한 Decoder-only architecture 입니다.


![](https://d2l.ai/_images/gpt-decoder-only.svg)
1. GPT는 transformer 아키텍처를 이용하므로, RNN처럼 순차적으로 토큰을 입력받지 않고 한번에 학습할 수 있습니다.
2. Causal Attention Mask를 이용하여, 다음 토큰의 어텐션을 받지 않도록 합니다.





## GPT는 어떻게 학습되는가?
1. GPT는 수백GB~수십TB규모의 대규모 웹 말뭉치를 이용하여 학습됩니다. 데이터 정제, 필터링, 중복 제거(Deduplication)을 거친 후, 텍스트를 통째로 모델에 넣어서 그대로 그 텍스트를 생성하게 만드는 단순한 방식입니다.
2. 하지만 텍스트마다 천차만별의 길이를 갖고있습니다. padding 기법을 사용할 수 있지만 더 효율적인 방법을 고안했습니다.바로 packing 기법입니다.

## Packing 기법이란?
서로 다른 길이의 텍스트를 스페셜 토큰을 사이로 이어붙여서 연장하는 기법입니다. 예를 들어 네개의 다른 길이를 가진 텍스트를 봅시다.

- 문장1:
아주대학교, 亞洲大學校, AJOU UNIVERSITY

- 문장2:
경기도 수원시에 있는 사립 종합대학교. 인간존중(人間尊重), 실사구시(實事求是), 세계일가(世界一家)라는 교육 이념을 추구하고 있다.

- 문장3: 아주대학교는 경기도의 중심인 수원시[5]에 위치해 있으며,  경기도 수원시의 대표 대학이자 4년제 사립 종합대학으로 인문사회계열, 이공계열, 의약계열과 대학병원이 모두 한 곳에 모여있는 단일캠퍼스이다.

- 문장4:
1971년 12월 한국과 프랑스 양국 간 '한·불기술 초급대학 설립에 관한 협정'이 체결되었다. 이 협정에 따라 1973년 3월 유신고속관광주식회사 박창원 사장은 유신학원을 발족했고 아주대학교의 전신인 아주공업초급대학을 설립하였다. 12월 아주공업초급대학은 아주공과대학으로 승격했다. 1977년 대우실업 사장을 맡았던 김우중 사장은 '교육사업으로 기업 이윤을 사회에 환원하고 시대가 요구하는 고급인력을 키우겠다.'는 목표를 갖고 사재 53억원을 출연해 학교법인 대우학원을 설립하여 아주공과대학을 인수하였다. 김우중 사장이 직접 사재를 출연해 인수한 아주공과대학은 1981년 3월 종합대학으로 승격하여 오늘날 아주대학교가 되었다.


띄어쓰기를 기준으로 토큰이 나눠진다고 가정하고, 사이에 넣는 스페셜 토큰은 `<eos>`라고 가정하겠습니다. 이제 이어붙이면


아주대학교, 亞洲大學校, AJOU UNIVERSITY `<eos>` 경기도 수원시에 있는 사립 종합대학교. 인간존중(人間尊重), 실사구시(實事求是), 세계일가(世界一家)라는 교육 이념을 추구하고 있다. `<eos> ` 아주대학교는 경기도의 중심인 수원시[5]에 위치해 있으며,  경기도 수원시의 대표 대학이자 4년제 사립 종합대학으로 인문사회계열, 이공계열, 의약계열과 대학병원이 모두 한 곳에 모여있는 단일캠퍼스이다. `<eos>` 1971년 12월 한국과 프랑스 양국 간 '한·불기술 초급대학 설립에 관한 협정'이 체결되었다. 이 협정에 따라 1973년 3월 유신고속관광주식회사 박창원 사장은 유신학원을 발족했고 아주대학교의 전신인 아주공업초급대학을 설립하였다. 12월 아주공업초급대학은 아주공과대학으로 승격했다. 1977년 대우실업 사장을 맡았던 김우중 사장은 '교육사업으로 기업 이윤을 사회에 환원하고 시대가 요구하는 고급인력을 키우겠다.'는 목표를 갖고 사재 53억원을 출연해 학교법인 대우학원을 설립하여 아주공과대학을 인수하였다. 김우중 사장이 직접 사재를 출연해 인수한 아주공과대학은 1981년 3월 종합대학으로 승격하여 오늘날 아주대학교가 되었다.


이제 이것을 모델이 생성 가능한 최대 시퀀스 길이로 나누면 하나의 학습 instance가 됩니다.


In [4]:
from transformers import GPT2Config, AutoTokenizer, AutoModelForCausalLM

In [5]:
# 우리는 사전 학습된 토크나이저를 쓰겠습니다.
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [6]:
text = "14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은 올라 직격타가 됐기 때문이다."
text2 = "고물가 부담으로 불가피하게 요금을 인상했지만, 요금은 여전히 지난달 경기도 평균 목욕비(9천638원)보다 저렴하다."
tokens = tokenizer.encode(text)
tokens2 = tokenizer.encode(text2)
print(len(tokens), len(tokens2))
packed = tokens + [tokenizer.eos_token_id] + tokens

print(tokenizer.decode(packed)) # 83

#
seq = 64
batch = packed[:seq]
print(tokenizer.decode(batch))

packed = packed[seq:]


text3 = "그는 “다른 곳은 1만원 안팎인데 우린 가격을 올려도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리지도 않았고 한동안 가격을 유지할 것”이라고 말했다."
tokens3 = tokenizer.encode(text3)
packed = packed + [tokenizer.eos_token_id] + tokens3

print(tokenizer.decode(packed))
print(len(packed))

packed = packed[seq:]

# seq len = 4
# [0 1 2 3] 4] = input_ids
# [1 2 3 4] = labels

56 27
14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은 올라 직격타가 됐기 때문이다.<|endoftext|> 14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은 올라 직격타가 됐기 때문이다.
14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은 올라 직격타가 됐기 때문이다.<|endoftext|> 14일 오후 수원특례시 장안
구에 위치한 ‘성일사우나’. 남효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은 올라 직격타가 됐기 때문이다.<|endoftext|> 그는 “다른 곳은 1만원 안팎인데 우린 가격을 올려도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리지도 않았고 한동안 가격을 유지할 것”이라고 말했다.
86


In [7]:
import torch

def pack_texts(items, max_seq_length):
  packed_tokens = []

  eos_token_id = tokenizer.eos_token_id

  for item in items:
    text =item["text"]
    tokens = tokenizer.encode(text)

    packed_tokens.extend(tokens)
    packed_tokens.append(eos_token_id)

    while len(packed_tokens) > max_seq_length:
      yield {
          "input_ids": torch.tensor(
              packed_tokens[ : max_seq_length],
              dtype=torch.int64
              ),
          "labels": torch.tensor(
              packed_tokens[1 : max_seq_length + 1],
              dtype=torch.int64
          )
      }
      packed_tokens = packed_tokens[max_seq_length:]


packed_tokens = pack_texts (
    [
        {"text": "14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은 올라 직격타가 됐기 때문이다."},
        {"text": "고물가 부담으로 불가피하게 요금을 인상했지만, 요금은 여전히 지난달 경기도 평균 목욕비(9천638원)보다 저렴하다."},
        {"text": "그는 “다른 곳은 1만원 안팎인데 우린 가격을 올려도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리지도 않았고 한동안 가격을 유지할 것”이라고 말했다."}
    ],
    16
)

for i, pack in enumerate(packed_tokens):
  print("#", i)
  print(len(pack["input_ids"]))
  print(tokenizer.decode(pack["input_ids"], skip_special_tokens=False))
  print(tokenizer.decode(pack["labels"], skip_special_tokens=False))

# 0
16
14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남
오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효
# 1
16
효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각
진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던
# 2
16
했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요
지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은
# 3
16
금은 올라 직격타가 됐기 때문이다.<|endoftext|> 고물가 부담으로 불가피하게 요
올라 직격타가 됐기 때문이다.<|endoftext|> 고물가 부담으로 불가피하게 요금을
# 4
16
금을 인상했지만, 요금은 여전히 지난달 경기도 평균 목욕비(9천638
인상했지만, 요금은 여전히 지난달 경기도 평균 목욕비(9천638원
# 5
16
원)보다 저렴하다.<|endoftext|> 그는 “다른 곳은 1만원 안팎인데 우린 가격을
)보다 저렴하다.<|endoftext|> 그는 “다른 곳은 1만원 안팎인데 우린 가격을 올려
# 6
16
올려도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리
도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리지도


예상 결과
```
# 0
14일 오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남
오후 수원특례시 장안구에 위치한 ‘성일사우나’. 남효
# 1
효진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각
진씨(64)가 운영하는 성일사우나의 상황은 코로나19가 심각했던
# 2
했던 지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요
지난해보다 더 안 좋아졌다. 손님은 줄고 상수도와 난방 요금은
# 3
금은 올라 직격타가 됐기 때문이다.<|endoftext|> 고물가 부담으로 불가피하게 요
올라 직격타가 됐기 때문이다.<|endoftext|> 고물가 부담으로 불가피하게 요금을
# 4
금을 인상했지만, 요금은 여전히 지난달 경기도 평균 목욕비(9천638
인상했지만, 요금은 여전히 지난달 경기도 평균 목욕비(9천638원
# 5
원)보다 저렴하다.<|endoftext|> 그는 “다른 곳은 1만원 안팎인데 우린 가격을
)보다 저렴하다.<|endoftext|> 그는 “다른 곳은 1만원 안팎인데 우린 가격을 올려
# 6
올려도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리
도 그보다 싸다”며 “착한가격업소니까 가격을 크게 올리지도
```

In [8]:
# 위키데이터를 가져와서 학습해보겠습니다.
# https://huggingface.co/datasets/heegyu/kowikitext

from datasets import load_dataset

# 데이터가 너무 많기 때문에, 한번에 가져오지 않고 스트리밍해서 가져옵니다.
dataset = load_dataset("heegyu/kowikitext", "20221001", split="train", streaming=True)

README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

kowikitext.py:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

In [9]:
from pprint import pprint

batch_size = 2
max_seq_length = 512

# IterableDataset 을 상속받아 구현해서, 텍스트를 순차적으로 스트리밍한 후 packing하여 학습 데이터로 보냅니다.
# 데이터가 너무 많아서 전체 길이를 알 수 없거나 스트리밍해서 불러오는 경우에 유용하게 사용됩니다
class TrainingDatasetStream(torch.utils.data.IterableDataset):
  def __init__(self, dataset, max_seq_length):
    self.dataset = dataset
    self.max_seq_length = max_seq_length

  def __iter__(self):
    return pack_texts(self.dataset, self.max_seq_length)

training_loader = torch.utils.data.DataLoader(
    TrainingDatasetStream(dataset, max_seq_length),
    batch_size=batch_size,
    )

batch = next(iter(training_loader))

pprint(batch)

pprint(tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=False))

{'input_ids': tensor([[15091,  9903,  9488,  ...,  7584, 17448,  9036],
        [14994, 39417,  9016,  ..., 27135, 15633, 12501]]),
 'labels': tensor([[ 9903,  9488,  8539,  ..., 17448,  9036, 14994],
        [39417,  9016, 10111,  ..., 15633, 12501,  9438]])}
['제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n'
 '생애.\n'
 '어린 시절.\n'
 '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n'
 '조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 '
 '이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n'
 '정계 입문.\n'
 '1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 '
 '낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 '
 '조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n'
 '대통령 재임.\n'
 '1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 '
 '누르고 당선되었다.\n'
 '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n'
 '외교 정책.\n'
 '카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위

## GPT 학습

In [10]:
print(tokenizer.vocab_size)
tokenizer.eos_token_id

51200


51200

In [11]:
config = GPT2Config (
    # 최대 시퀀스 길이 512
    n_ctx = max_seq_length,
    n_positions = max_seq_length,

    # 어휘사전 크기 (tokenizer 보다 커야함!)
    vocab_size = tokenizer.vocab_size + 1,

    n_embd = 128,

    n_head = 2,
    n_layer = 2,

    # dropout
    resid_pdrop = 0.1,
)
model = AutoModelForCausalLM.from_config(config)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51201, 128)
    (wpe): Embedding(512, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=384, nx=128)
          (c_proj): Conv1D(nf=128, nx=128)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=512, nx=128)
          (c_proj): Conv1D(nf=128, nx=512)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=51201, bias=False)
)

1. 언어모델의 크기를 산정할 때는, 일반적으로 임베딩 레이어, 마지막 LM Head의 크기를 제외합니다.
2. GPT의 경우 마지막 LM head 대신 embedding layer를 LM head 대신 사용해서 파라미터 수를 줄이는 경우가 있습니다 (tied weight). GPT2는 tied weights를 기본으로 사용합니다. 때문에 유사한 다른 아키텍처(GPT-Neo, GPT-NeoX) 보다 전체 파라미터 수가 줄어듭니다.

In [12]:
def count_parameters(model):
  params = sum(p.numel() for name, p in model.named_parameters())
  return params

print("total params", count_parameters(model))
print("model params", count_parameters(model.transformer.h))

total params 7016064
model params 396544


In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

사전학습은 일반적으로 학습 데이터가 너무 많아서, 1 에폭 학습도 하지 못하는 경우가 많습니다. 이럴 때는 step 단위로 학습을 하게 됩니다.

In [ ]:
from tqdm.auto import tqdm
import torch.nn.functional as F



def change_dict_device(x):
  return {k: v.to(device) for k, v in x.items()}

running_loss = 0.
last_loss = 0.
log_freq = 50
i = 0

progress = tqdm(training_loader, desc=f"Training")

# 1000 스텝만 학습하기
train_steps = 1000

model.train()
while True:

  for batch in training_loader:
    i += 1

    optimizer.zero_grad()
    batch = change_dict_device(batch)

    logits = model(batch['input_ids']).logits

    logits = logits.view(-1, logits.shape[-1]) # [batch_size * sequence_length, vocab_size]
    labels = batch["labels"].view(-1) # [batch_size * sequence_length, vocab_size]

    loss = F.cross_entropy(logits, labels, ignore_index=-100)
    loss.backward()

    optimizer.step()

    running_loss += loss.item()
    if (i + 1) % log_freq == 0:
      last_loss = running_loss / log_freq
      print('batch {} loss: {}'.format(i + 1, last_loss))
      running_loss = 0.

    progress.update(1)

    if i >= train_steps:
      break

  if i >= train_steps:
    break

Training: 0it [00:00, ?it/s]

batch 50 loss: 8.608162841796876
batch 100 loss: 8.604558744430541
batch 150 loss: 9.166077146530151
batch 200 loss: 9.174205436706544
batch 250 loss: 8.815451555252075
batch 300 loss: 9.01519121170044
batch 350 loss: 9.192004737854004
batch 400 loss: 9.029533596038819
batch 450 loss: 8.982937126159667
batch 500 loss: 9.248815746307374
batch 550 loss: 9.132542018890382
batch 600 loss: 8.913141870498658
batch 650 loss: 9.125893087387086
batch 700 loss: 9.172744464874267
batch 750 loss: 8.956591711044311
batch 800 loss: 8.916443624496459
batch 850 loss: 9.00273769378662
batch 900 loss: 8.937515707015992
batch 950 loss: 8.856464595794678
batch 1000 loss: 8.637866821289062


In [ ]:
# 한번 테스트!
prefix = tokenizer(["오늘의 날씨는 "], return_tensors="pt").to(device)
outputs = model.generate(prefix['input_ids'], max_new_tokens=128, do_sample=True)
print(tokenizer.decode(outputs[0]))


### 그래디언트 누적
모델의 크기가 크면 큰 배치 크기를 가질 수 없습니다. 때문에 그래디언트 누적(Gradient Accumulation) 기법을 사용합니다

#### 예시:
```
total_batch = 128 # 내가 원하는 배치 크기
step_batch = 2 # 실제 모델이 한번에 처리하는 배치 크기

acc_steps = total_batch // step_batch # 나머지가 생기면 안됩니다.

# 학습 루프
for i, batch in enumerate(train_dataloader):
  output = model(batch)
  loss = ...
  loss.backward()

  # backward를 acc_steps번 할 때마다 optimizer를 step합니다
  if (i + 1) % acc_steps == 0:
    optimizer.step()
    optimizer.zero_grad()

```

### 그래디언트 클리핑(Gradient Clipping)
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FsPJBj%2FbtqJjuE2KtK%2FPgEuZIBj5pkL4Mmb0K8P9k%2Fimg.png)

큰 모델의 학습은 일반적으로 gradient vanishing, gradient clipping을 방지하기 위해, gradient norm을 제한하는 gradient clipping 기법을 사용합니다.

```
max_norm = 1.0
# step 직전에 해주기
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
optimizer.step()
```

# 몇가지 생각할 만한 주제
- Q1: 학습 데이터는 일반적으로 섞어야 하는데, streaming할 때는 어떻게 섞어야 할까요?
- Q2: packing은 데이터 효율을 높아주지만 이런식으로 학습해도 정말 괜찮을까요...?
- Q3: `<eos>` 토큰을 기준으로 packing을 하면, 한 instnace 내의 마지막 문장은 `<eos>`가 나오기 전에 끊어지게 됩니다. 이런 경우는 어떻게 처리할까요?

### Tied weights 기법에 대한 자세한 설명
GPT2 아키텍처는 크게 아래 3가지로 나뉩니다.
1. 임베딩: 토큰 id를 임베딩으로
2. transformer block 레이어
3. LM head: transformer block을 거친 임베딩을 다시 토큰으로 바꾸기. 정확히는 마지막 임베딩이 vocab size만큼의 토큰 중 어떤 토큰일지 분류해내는 task

![](https://www.researchgate.net/publication/349630764/figure/fig3/AS:999014610788354@1615195052671/Detail-of-the-embedding-layer-of-the-NN-implementing-the-Encoding-model-used-for-the.png)
- 임베딩 레이어는 위 사진처럼 행렬에서 특정 index의 row를 가져오는 것과 같습니다.
- 그렇다면 임베딩을 위 행렬의 전치행렬과 다시 곱하면 어떻게 될까요?
- 이는 곳 임베딩과 행렬 내의 모든 row와의 코사인 유사도가 됩니다.
- 그렇다면 원래 임베딩과 높은 유사도가 나오겠죠?
- 이 원리를 이용하여 LM head에서 embedding 레이어의 값을 사용하게 할 수 있습니다.
- 마지막 LM head는 실제로는 linear 레이어, 즉 행렬의 곱셈입니다. 이때 행렬곱은 벡터간의 내적연산으로 볼 수 있으며, 코사인 유사도로 볼 수 있죠